In [1]:
from ucimlrepo import fetch_ucirepo 

In [2]:
# fetch dataset 
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697) 
  
# data (as pandas dataframes) 
X = predict_students_dropout_and_academic_success.data.features 
y = predict_students_dropout_and_academic_success.data.targets 


In [3]:
X.shape

(4424, 36)

In [4]:
X.columns

Index(['Marital Status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd s

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 36 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital Status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [6]:
y_copy = y.copy()

# Update the 'Target' column in the copy
y_copy['Target'] = y_copy['Target'].map({
    'Dropout': 0,
    'Enrolled': 1,
    'Graduate': 2
})

# Now 'y_copy' contains the modified 'Target' column
# If you want to update the original 'y' DataFrame, you can do:
y.loc[:, 'Target'] = y_copy['Target']

/tmp/ipykernel_47258/1178644789.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[:, 'Target'] = y_copy['Target']
/tmp/ipykernel_47258/1178644789.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  y.loc[:, 'Target'] = y_copy['Target']


In [7]:
import numpy as np

In [8]:
student_x = X
student_y = y

In [9]:
from sklearn.model_selection import train_test_split
student_x_train, student_x_test, student_y_train, student_y_test = train_test_split(student_x, student_y, test_size=1/3, random_state=42)
student_X_train, student_X_val, student_Y_train, student_y_val = train_test_split(student_x_train, student_y_train, test_size=1/3, random_state=42)

In [10]:
import util
from importlib import reload
reload(util)
from util import Utils

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [11]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

best_student_model = util.Utils.find_best_rf_parameters(student_X_train, student_Y_train.values.ravel(), student_X_val, student_y_val.values.ravel())

|   n_estimators | criterion   |   max_depth | max_features   |   score in validation set |
|---------------:|:------------|------------:|:---------------|--------------------------:|
|            100 | gini        |         100 | log2           |                  0.772126 |
|             50 | gini        |         100 | log2           |                  0.765005 |
|             10 | gini        |         100 | log2           |                  0.759919 |
|            100 | gini        |         100 | sqrt           |                  0.770092 |
|             50 | gini        |         100 | sqrt           |                  0.768057 |
|             10 | gini        |         100 | sqrt           |                  0.741607 |
|            100 | gini        |          50 | log2           |                  0.769074 |
|             50 | gini        |          50 | log2           |                  0.770092 |
|             10 | gini        |          50 | log2           |                 

In [15]:
Utils.accuracy_measure_rf(student_x_train, student_y_train.values.ravel(), student_x_test, student_y_test.values.ravel(), n_estimators = 100, criterion = 'entropy', max_depth = 50, max_features = 'sqrt')

--- 0.6734986305236816 seconds ---


(0.7566101694915254,
 (array([0.80128205, 0.53435115, 0.76598174]),
  array([0.76687117, 0.25830258, 0.93846154]),
  array([0.78369906, 0.34825871, 0.84349466]),
  array([489, 271, 715])))

In [14]:
best_xgb_model = Utils.find_best_xgb_parameters(student_X_train, student_Y_train, student_X_val, student_y_val)

|   n_estimators |   learning_rate |   max_depth |   subsample |   colsample_bytree |   score in validation set |
|---------------:|----------------:|------------:|------------:|-------------------:|--------------------------:|
|            100 |            0.5  |          50 |         0.8 |                1   |                  0.765005 |
|            100 |            0.5  |          50 |         0.5 |                1   |                  0.76297  |
|             50 |            0.5  |          50 |         0.8 |                1   |                  0.763988 |
|             50 |            0.5  |          50 |         0.5 |                1   |                  0.757884 |
|             10 |            0.5  |          50 |         0.8 |                1   |                  0.760936 |
|             10 |            0.5  |          50 |         0.5 |                1   |                  0.750763 |
|            100 |            0.5  |          10 |         0.8 |                1   |   

In [16]:
Utils.accuracy_measure_xgb(student_x_train, student_y_train.values.ravel(), student_x_test, student_y_test.values.ravel(), n_estimators = 100, learning_rate = 0.1, max_depth = 50, subsample = 0.8, colsample_bytree = 1)

--- 0.7991156578063965 seconds ---


(0.7647457627118645,
 (array([0.80215054, 0.55421687, 0.78554502]),
  array([0.76278119, 0.33948339, 0.92727273]),
  array([0.78197065, 0.42105263, 0.85054522]),
  array([489, 271, 715])))